In [1]:
#!pip install selenium
#!pip install geonamescache
from helper import *
import selenium
import tweepy
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from IPython.display import clear_output #clears output
import datetime
import time

API_Key ='irXxqVrXin1KsDRM9g6HdF8p8'
API_Key_Secret = 'sLQvb8voyrTDz5S93vlt5jE5RlMi8xBuOQtSq2pgERe2QVawIC'

Bearer_Token = 'AAAAAAAAAAAAAAAAAAAAACsVlAEAAAAADib49u9Asa6Hbts8UwBEdtc0dsk%3D1d8lVaySeRg1CpJQ6HDLSwK6Mj9wJiptj49PAdwMuvW95zJoar'

Access_Token = '1122492746210660352-COptyz3RtF1AVokHw3027iOdbsZZl1'
Access_Token_Secret = 'slfBz8pcgCrUmewZgSgVKsmIFhfCcFuinAx16q5eh1Vvr'

In [2]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
# set access to user's access key and access secret
auth.set_access_token(Access_Token, Access_Token_Secret)
# calling the api
api = tweepy.API(auth)

Client = tweepy.Client(bearer_token=Bearer_Token) 


In [3]:
'''
#tweets_list = api.search_30_day(label = '30day',query = '#blender', fromDate = FROM_DATE,toDate = TO_DATE,maxResults = 10,tweet_mode = "extended")
#tweets_list = tweepy.Cursor(api.search_tweets, "biden", count=100).items(100)


start_date = datetime.datetime.now() - datetime.timedelta(days=14)
end_date = start_date + datetime.timedelta(days=7)

start_date = datetime.datetime.now()
end_date = start_date + datetime.timedelta(days=1)
'''

# Search for tweets within the target countries
usa = '96683cc9126741d1'
canada = "3376992a082d67c7"
great_britain = '6416b8512febefc9'
Australia = '3f14ce28dc7c4566'
new_zealand = "0b039ce18ddf1cb9"

tweets_list = []
#ALL_pages = tweepy.Cursor(api.search_tweets, q="since:{} until:{}".format(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")), lang="en", tweet_mode="extended", count = 100).pages(50)
max_count_4_each_country = 1500

#add usa
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ usa + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
time.sleep(10)

#add Canada
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ canada + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
time.sleep(10)

#add "Great" Britian
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ great_britain + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
time.sleep(10)

#add Australia
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ Australia + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
time.sleep(10)

#add new zealand
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ new_zealand + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)





'''
#return this if you are using pages again!!!!!
for page in ALL_pages:
    tweets_list.extend(page)
#tweets_list = [t for t in tweets_list if t.in_reply_to_status_id == None]    
'''
print('done')

done


In [4]:
tweets_list = [t for t in tweets_list]
len(tweets_list)

7500

In [5]:
#search_tweets(q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)

#user data

DRIVER = webdriver.Chrome()#the chrome

user_name = [];
location = [];
age = [];
gender = [];
followers = [];
following = [];
verified = []
joined = [] #when they joined
tweet_count_user = []

#tweet data
tweet = [];
word_count = [];
lang = []
likes = []
retweet_count = []
is_quote_status = []
date = []
tweet_id = []
hashtags = []
trending_for_area = []

users = [];

print('loading users')

arr = []
i = 0

for t in tweets_list:
    tweet.append(t.full_text);
    users.append(t.user);
    word_count.append(countWordsInString(t.full_text))
    lang.append(t.lang)
    likes.append(t.favorite_count) 
    retweet_count.append(t.retweet_count)
    is_quote_status.append(t.is_quote_status)
    date.append(t.created_at.isoformat()[:10])
    tweet_id.append(t.id)
    hashtags.append(t.entities['hashtags'])
    location.append([t.place.name,t.place.country])
    arr.append([t.place.name,t.place.country])
    woe = get_WOEID(t.place.name,t.place.country)
    try:
        trending_for_area.append(api.get_place_trends(woe))
    except:
        trending_for_area.append(np.nan)
        
for x in users:
    if i%25 == 0:
        print("waiting")
        time.sleep(6)
    
    j = x._json
    #opens the page and waits for it to load
    screen_name = j['screen_name']
    #country = check_country_real(j['location'])
    
    DRIVER.get('https://twitter.com/' + screen_name) 
    
    #get the easy stuff
    user_name.append(screen_name)
    following.append(j['friends_count']);
    followers.append(j['followers_count']);
    #location.append(country);
    joined.append(x.created_at.isoformat()[:10])
    tweet_count_user.append(x.statuses_count)
    verified.append(x.verified)
    
    
    
    tmp = getAge(DRIVER)#gets the age
    if(tmp == 2023 - x.created_at.year): #checks if the age is the same as the accounts age
        age.append(np.nan)
    else:
        age.append(tmp)
    
    
    gender.append(getGender(j['description'],j['location'],j['name']));
    
    ##this part simply displays at the bottom how many profiles were checked
    clear_output(wait=True) #clears the output
    
    i+=1
    print(i)

print('done')

7500
done


In [6]:
max = len(users)

In [7]:
dick = {"name":user_name[:max],
        "age":age[:max],
        'location':location[:max],
        "gender":gender[:max],
        'join_date':joined[:max],
        'total tweets': tweet_count_user[:max],
        "followers":followers[:max],
        "following":following[:max],
        'verified' : verified[:max],
        'tweet':tweet[:max],
        'likes':likes[:max],
        'retweets':retweet_count[:max],
        'date':date[:max],
        'word count': word_count[:max],
        'launguage': lang[:max],
        'is quote':is_quote_status[:max],
        'hashtags':hashtags[:max],
        'trending': trending_for_area[:max],
        'tweet id':tweet_id[:max]
       }

df = pd.DataFrame(dick)

In [8]:
df

,name,age,location,gender,join_date,total tweets,followers,following,verified,tweet,likes,retweets,date,word count,launguage,is quote,hashtags,trending,tweet id
0,Superfan_K,NaN,"[Indianapolis, United States]",None,2017-06-24,128288,1241,1218,False,YES https://t.co/b7FMYonCwh,0,0,2023-01-11,2,und,True,[],NaN,1612991908321849349
1,SangerEsports,NaN,"[Shady Shores, United States]",None,2022-01-08,154,153,411,False,Hey Oddfalls! You using all those chairs buddy...,0,0,2023-01-11,17,en,False,"[{'text': 'esportsfun', 'indices': [79, 90]}]",NaN,1612991908162449409
2,thegreenman1949,NaN,"[Tucson, United States]",None,2020-08-28,266441,4911,5403,False,Remember that the Democrats are Satan worshipp...,0,0,2023-01-11,12,en,True,[],NaN,1612991907617210368
3,tinseltine,NaN,"[Jenkintown, United States]",None,2009-07-15,23147,1803,1516,False,#GoldenGlobes We @PhilaFCC felt the same! \nht...,0,0,2023-01-11,10,en,True,"[{'text': 'GoldenGlobes', 'indices': [0, 13]},...",NaN,1612991906421657600
4,LaurieThrasher,NaN,"[Baltimore, United States]",None,2013-10-02,108297,45951,44694,False,I was ready to conquer the world! 🗺️ https://t...,0,0,2023-01-11,9,en,True,[],NaN,1612991904064409600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,artbyira,NaN,"[Queenstown-Lakes District, New Zealand]",None,2009-12-14,14195,365,1756,False,Just posted a photo @ Arrowtown https://t.co/f...,0,0,2023-01-09,7,en,False,[],NaN,1612285510138957824
7496,urbanmaaori,NaN,"[Christchurch City, New Zealand]",None,2022-12-21,433,20,243,False,'Twas a thief said the last kind word to Chris...,0,0,2023-01-09,22,en,True,[],NaN,1612284009635250178
7497,imnmoz,NaN,"[Auckland, New Zealand]",None,2009-09-03,146831,882,106,False,Env ตัวเองเจ้งถาวร shared env ก็เจ้ง อิควัย ปี...,0,0,2023-01-09,7,th,False,[],NaN,1612283981738971136
7498,everylotakl,NaN,"[Auckland, New Zealand]",None,2020-01-16,37160,804,55,False,"111A Gardner Avenue, New Lynn\n\nEstimated val...",1,0,2023-01-09,13,en,False,[],NaN,1612283067833671680


In [9]:
DRIVER.quit()

In [10]:
#df.to_csv('7k.csv')

In [11]:
woe = get_WOEID(arr[1][0],arr[1][1])
api.get_place_trends(1)

[{'trends': [{'name': '#GoldenGlobes2023',
    'url': 'http://twitter.com/search?q=%23GoldenGlobes2023',
    'promoted_content': None,
    'query': '%23GoldenGlobes2023',
    'tweet_volume': 164199},
   {'name': 'Twitter Blue',
    'url': 'http://twitter.com/search?q=%22Twitter+Blue%22',
    'promoted_content': None,
    'query': '%22Twitter+Blue%22',
    'tweet_volume': 215682},
   {'name': 'Warriors',
    'url': 'http://twitter.com/search?q=Warriors',
    'promoted_content': None,
    'query': 'Warriors',
    'tweet_volume': 61915},
   {'name': '#NaatuNaatu',
    'url': 'http://twitter.com/search?q=%23NaatuNaatu',
    'promoted_content': None,
    'query': '%23NaatuNaatu',
    'tweet_volume': 104063},
   {'name': 'Jennifer Coolidge',
    'url': 'http://twitter.com/search?q=%22Jennifer+Coolidge%22',
    'promoted_content': None,
    'query': '%22Jennifer+Coolidge%22',
    'tweet_volume': 26951},
   {'name': 'Vince',
    'url': 'http://twitter.com/search?q=Vince',
    'promoted_content